In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from glmsingle_utils import (
    load_results, 
    plot_results,
    save_results
)

In [ ]:
# Enter the path to the Kamitani dataset, example: X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\fmriprep-20.2.4
dataset_path = Path(input())

In [ ]:
# Load layouts, this will take about a minute because pybids is slow

t = time.time()
dataset_layout = bids.BIDSLayout(dataset_path)
print(time.time() - t)

In [ ]:
# There were 15 sessions to acquire training data in this dataset
# These three groups of sessions presented the same images in the same runs
# Each image in the training set was presented exactly 5 times
# It is better to run GLMsingle on the same run from one of these groups so cross validation is possible
session_groups = {
    'A': [f'perceptionNaturalImageTraining{i:02}' for i in (1, 4, 7, 10, 13)],
    'B': [f'perceptionNaturalImageTraining{i:02}' for i in (2, 5, 8, 11, 14)],
    'C': [f'perceptionNaturalImageTraining{i:02}' for i in (3, 6, 9, 12, 15)],
}

In [ ]:
sessions = session_groups['A'] # select one or more session groups
subject = '02' # ['01', '02', '03']
space = 'T1w'
run_ids = [1] # one or more runs
tr = 2.

run_images = []
events_dfs = []
for session in sessions:
    for run_id in run_ids:

        bids_image = dataset_layout.get(
            subject=subject, 
            session=session, 
            space=space, 
            run=run_id, 
            desc='preproc', 
            extension='nii.gz'
        )[0]

        events_files = dataset_layout.get(
            subject=subject, 
            session=session, 
            run=run_id, 
            extension='tsv', 
            suffix='events',
        )
        events_df = pd.read_csv(events_files[0].path, sep='\t',)# dtype={'stimulus_id': str})

        events_dfs.append(events_df)
        run_images.append(bids_image.get_image())

In [ ]:
# Load the fmri mask. 
# fmriPrep outputs one mask for every functional run, but they are mostly identical
# This will grab the mask from the very first one
mask_image = dataset_layout.get(
    subject=subject, 
    session=session, 
    space=space, 
    run=run_id, 
    desc='brain', 
    extension='nii.gz'
)[0].get_image()
fmri_mask = mask_image.get_fdata().astype(bool)
H, W, D = fmri_mask.shape

# Load the fmri data and apply the mask
fmri_batch = []
for run_image in tqdm(run_images):
    fmri_data = run_image.get_fdata()
    fmri_data = fmri_data[fmri_mask]
    fmri_batch.append(fmri_data)

In [ ]:
# Cell for visualizing the fMRI runs
all_runs = np.concatenate(fmri_batch)
mean, std = all_runs.mean(), all_runs.std()

show_volume = np.zeros_like(fmri_mask, dtype=float)
_, T = fmri_batch[0].shape

@interact(run_id=(0, len(fmri_batch)-1), d=(0, D-1), t=(0, T-1))
def show(run_id, d, t):
    show_volume[fmri_mask] = fmri_batch[run_id][:, t]
    plt.figure(figsize=(8, 8))
    plt.imshow(show_volume[:, :, d], cmap='gray', vmin=mean-2*std, vmax=mean+2*std)
    plt.show()

In [ ]:
conditions

In [ ]:
# Create the design matrices

# Collect all conditions
conditions = []
for events_df in events_dfs:
    for i, event in events_df.iterrows():
        if int(event['event_type']) != 1:
            continue
        conditions.append(event['stimulus_id'])
conditions = list(set(conditions))
conditions.sort()
conditions = {condition: i  for i, condition in enumerate(conditions)}
C = len(conditions)

T = fmri_batch[0].shape[-1]

# Construct the design matrices
design_batch = []
for run_id, events_df in enumerate(events_dfs):
    design_matrix = np.zeros(shape=(T, C))
    
    for i, event in events_df.iterrows():
        if int(event['event_type']) != 1:
            continue
        condition_name = event['stimulus_id']
        c = conditions[condition_name]
        t = round(event['onset'] / tr)
        design_matrix[t, c] = 1
    design_batch.append(design_matrix)
    
@interact(run_id=(0, len(design_batch)-1))
def show(run_id):
    design_matrix = design_batch[run_id]
    plt.figure(figsize=(12, 12))
    plt.xlabel('Conditions')
    plt.ylabel('TRs')
    plt.imshow(design_matrix)

In [ ]:
# Load previous results
runs_path = dataset_path / f'../glmsingle/'
previous_runs = [p.name for p in runs_path.iterdir()]
print(previous_runs)

run_name = 'run2'
output_path = runs_path / run_name
results_glmsingle = load_results(output_path)

In [ ]:
# Or run GLMsingle from scratch

#break # set a new run name before running to avoid overwriting old results
run_name = 'run_4'

glmsingle_obj = GLM_single(dict(
    wantlibrary=1,
    wantglmdenoise=1,
    wantfracridge=1,
    wantfileoutputs=[1,1,1,1],
    wantmemoryoutputs=[1,1,1,1],
))

pprint(glmsingle_obj.params)

output_path = dataset_path / f'derivatives_TC2See_prdgm/glmsingle/{run_name}'
results_glmsingle = glmsingle_obj.fit(
    design=design_batch,
    data=fmri_batch,
    stimdur=8,
    tr=tr,
    outputdir=str(output_path),
)

In [ ]:
# Save nifti files for results
save_results(results_glmsingle, output_path, fmri_mask)

In [ ]:
# Inspect the results at different layers
@interact(d_layer=(0, D-1))
def plot_results_layer(d_layer):
    plot_results(results_glmsingle, fmri_mask, d_layer)